# Analyse du dataset de Plonger 

## Importation des packadges

In [10]:
import numpy as np
import pandas as pd

## Resumé du dataset

Lecture du csv

In [11]:
dataset = pd.read_csv('AIDA_Results_IA_Institut.csv')

Affichage des 10 premières lignes

In [12]:
dataset.head(10)

,Start,Diver,Gender,Discipline,Line,Official Top,AP,RP,Card,Points,Remarks,Title Event,Event Type,Day,Category Event
0,1,Tasos Grillakis (GRC),M,FIM,NaN,00:00,33,23 m,YELLOW,12.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
1,2,Antonis Papantonatos (GRC),M,FIM,NaN,00:00,55,47 m,YELLOW,38.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
2,3,Dimitris Koumoulos (GRC),M,CNF,NaN,00:00,55,55 m,WHITE,55.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
3,4,Christos Papadopoulos (GRC),M,CWT,NaN,00:00,55,55 m,WHITE,55.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
4,5,Anna Chalari (GRC),F,CWT,NaN,00:00,15,15 m,WHITE,15.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
5,6,Alkisti Vouraki (GRC),F,CWT,NaN,00:00,16,16 m,WHITE,16.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
6,7,Stavroula Kontorousi (GRC),F,CWT,NaN,00:00,20,20 m,WHITE,20.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
7,8,Yuliya Ilkevich (BLR),F,CWT,NaN,00:00,40,31 m,YELLOW,21.0,"SHORT, TAG",Depth Event 2016,Depth Competition,2016-07-17,other
8,9,Dimitris Tsigkounis (GRC),M,CWT,NaN,00:00,40,26 m,YELLOW,11.0,"SHORT, TAG",Depth Event 2016,Depth Competition,2016-07-17,other
9,10,Emmanouil Marneros (GRC),M,CWT,NaN,00:00,47,47 m,WHITE,47.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other


In [13]:
dataset.shape

(26841, 15)

In [14]:
dataset.isna().sum()

Start                 0
Diver                 0
Gender                0
Discipline            0
Line              22147
Official Top          0
AP                    0
RP                    0
Card                  0
Points                0
Remarks               5
Title Event           0
Event Type            0
Day                   0
Category Event        0
dtype: int64

In [16]:
non_numeric_df = dataset[['Diver','Gender', 'Discipline', 'Card', 'Remarks', 'Title Event', 'Event Type', 'Day', 'Category Event']]
numeric_df = dataset[['Official Top', 'AP', 'RP', 'Points']]

In [17]:
non_numeric_df

,Diver,Gender,Discipline,Card,Remarks,Title Event,Event Type,Day,Category Event
0,Tasos Grillakis (GRC),M,FIM,YELLOW,-,Depth Event 2016,Depth Competition,2016-07-17,other
1,Antonis Papantonatos (GRC),M,FIM,YELLOW,-,Depth Event 2016,Depth Competition,2016-07-17,other
2,Dimitris Koumoulos (GRC),M,CNF,WHITE,-,Depth Event 2016,Depth Competition,2016-07-17,other
3,Christos Papadopoulos (GRC),M,CWT,WHITE,OK,Depth Event 2016,Depth Competition,2016-07-17,other
4,Anna Chalari (GRC),F,CWT,WHITE,OK,Depth Event 2016,Depth Competition,2016-07-17,other
...,...,...,...,...,...,...,...,...,...
26836,Karine Le Flanchec (FRA),F,CWTB,WHITE,Ok,The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other
26837,Anne-Sophie Passalboni (FRA),F,CWT,WHITE,Ok,The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other
26838,Clementine Marie (FRA),F,CNF,WHITE,Ok,The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other
26839,Eisve Treciakauskaite (LTU),F,CWT,YELLOW,"No tag,under ap",The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other


In [18]:
numeric_df

,Official Top,AP,RP,Points
0,00:00,33,23 m,12.0
1,00:00,55,47 m,38.0
2,00:00,55,55 m,55.0
3,00:00,55,55 m,55.0
4,00:00,15,15 m,15.0
...,...,...,...,...
26836,09:40,57,57 m,57
26837,09:50,52,52 m,52
26838,10:00,40,40 m,40
26839,10:10,33,28 m,22


In [12]:
# Extrait les 5 derniers caractères de la colonne 'Nom_de_la_colonne'
dataset['Nom_des_Pays'] = dataset['Diver'].str[-4:-1]

# Affiche le DataFrame mis à jour
dataset

,Start,Diver,Gender,Discipline,Line,Official Top,AP,RP,Card,Points,Remarks,Title Event,Event Type,Day,Category Event,Nom_des_Pays
0,1,Tasos Grillakis (GRC),M,FIM,NaN,00:00,33,23 m,YELLOW,12.0,-,Depth Event 2016,Depth Competition,2016-07-17,other,GRC
1,2,Antonis Papantonatos (GRC),M,FIM,NaN,00:00,55,47 m,YELLOW,38.0,-,Depth Event 2016,Depth Competition,2016-07-17,other,GRC
2,3,Dimitris Koumoulos (GRC),M,CNF,NaN,00:00,55,55 m,WHITE,55.0,-,Depth Event 2016,Depth Competition,2016-07-17,other,GRC
3,4,Christos Papadopoulos (GRC),M,CWT,NaN,00:00,55,55 m,WHITE,55.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other,GRC
4,5,Anna Chalari (GRC),F,CWT,NaN,00:00,15,15 m,WHITE,15.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other,GRC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26836,7,Karine Le Flanchec (FRA),F,CWTB,1.0,09:40,57,57 m,WHITE,57,Ok,The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other,FRA
26837,8,Anne-Sophie Passalboni (FRA),F,CWT,1.0,09:50,52,52 m,WHITE,52,Ok,The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other,FRA
26838,9,Clementine Marie (FRA),F,CNF,1.0,10:00,40,40 m,WHITE,40,Ok,The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other,FRA
26839,10,Eisve Treciakauskaite (LTU),F,CWT,1.0,10:10,33,28 m,YELLOW,22,"No tag,under ap",The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other,LTU
